<a href="https://colab.research.google.com/github/jongnnis/Ensemble/blob/main/boosting_stacking_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score

In [ ]:
iris = load_iris()

- sepal length in cm: 꽃받침의 길이
- sepal width in cm: 꽃받침의 너비
- petal length in cm: 꽃잎의 길이
- petal width in cm: 꽃잎의 너비

In [ ]:
data = iris['data']
feature_names = iris['feature_names']

df_iris = pd.DataFrame(data, columns=feature_names)
df_iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
target = iris['target']
df_iris['target'] = target
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split (df_iris.drop('target',1), df_iris['target'], test_size=0.2, random_state=2023)

<ipython-input-11-70dfa2112376>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train, X_test, y_train, y_test = train_test_split (df_iris.drop('target',1), df_iris['target'], test_size=0.2, random_state=2023)


In [ ]:
X_train.shape, y_train.shape

((120, 4), (120,))

In [ ]:
X_test.shape, y_test.shape

((30, 4), (30,))

# 1. 부스팅(Boosting)

### 1-1. AdaBoosting

In [ ]:
# 약한 학습기 정의
weak_learner = DecisionTreeClassifier(max_depth=1)

In [ ]:
# AdaBoost 모델 생성
adaboost_model = AdaBoostClassifier(base_estimator=weak_learner, n_estimators=10, random_state=2023)
# base_estimator: 약한 학습기 지정  # n_estimators: 학습기 개수 지정

In [ ]:
# 모델 학습
adaboost_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred1 = adaboost_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
# 정확도 평가
accuracy = accuracy_score(y_test, pred1)
print(f'정확도: {accuracy}')

정확도: 0.9666666666666667


In [ ]:
# 사용된 모형 개수
len(adaboost_model.estimators_)

10

- AdaBoosting 장단점
  * 장점: 과적합의 영향을 덜 받는다. 구현이 쉽다.
  * 단점: 이상치에 민감하다. 해석이 어렵다.

### 1-2. Gradient Boosting

In [ ]:
# Gradient Boosting 모델 생성
gradient_boost_model = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=2023)

In [ ]:
# 모델 학습
gradient_boost_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred2 = gradient_boost_model.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, pred2)
print(f'정확도: {accuracy}')

정확도: 1.0


- Gradient Boosting 장단점
  * 장점: 구현이 쉽다. 정확도가 좋다
  * 단점: 과적합 발생 가능성이 크다. 메모리 문제가 있다. 해석이 어렵다.

### 1-3. XGBoost

In [ ]:
# XGBoost 모델 생성
xgboost_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(df_iris['target'])), random_state=2023)
# objective: 손실 함수를 지정하는 매개변수  # 'multi:softmax': 소프트맥스 손실을 최적화   # num_class: 다중 클래스 분류 문제에서 클래스의 개수

In [ ]:
# 모델 학습
xgboost_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred3 = xgboost_model.predict(X_test)

In [ ]:
# 정확도 평가
accuracy = accuracy_score(y_test, pred3)
print(f'정확도: {accuracy}')

정확도: 1.0


- XGBoost 장단점
  * 장점: 과적합 방지가 되어있다. 예측 성능이 좋다.
  * 단점: 작은 데이터에 대하여 과적합 발생 가능성이 있다. 해석이 어렵다.

# 2. 스태킹

### 2-1. 기본 스태킹

In [ ]:
# 기본 모델들 정의
base_models = [
    ('rf', RandomForestClassifier(n_estimators=50, random_state=2023)),
    ('gb', GradientBoostingClassifier(n_estimators=50, random_state=2023))
]

In [ ]:
# 메타 모델 정의
meta_model = LogisticRegression()

# 스태킹 모델 정의
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

In [ ]:
# 모델 학습
stacking_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred4 = stacking_model.predict(X_test)

In [ ]:
# 정확도 평가
accuracy = accuracy_score(y_test, pred4)
print(f'정확도: {accuracy}')

정확도: 1.0


### 2-2. 블렌딩(스태킹기법)

In [ ]:
# 기본 모델들 정의
base_model1 = RandomForestClassifier(n_estimators=50, random_state=2023)
base_model2 = GradientBoostingClassifier(n_estimators=50, random_state=2023)

In [ ]:
# 메타 모델 정의
meta_model = LogisticRegression()

In [ ]:
# 각 기본 모델을 훈련시키고 예측 결과 저장
base_model1.fit(X_train, y_train)
pred_1 = base_model1.predict(X_test)

base_model2.fit(X_train, y_train)
pred_2 = base_model2.predict(X_test)

In [ ]:
# 메타 모델을 위한 훈련 데이터 생성
meta_train_data = np.column_stack((pred_1, pred_2))

# 메타 모델 훈련
meta_model.fit(meta_train_data, y_test)

LogisticRegression()

In [ ]:
# 각 기본 모델을 사용하여 예측
pred1_final = base_model1.predict(X_test)
pred2_final = base_model2.predict(X_test)

# 메타 모델에 입력으로 사용할 데이터 생성
meta_test_data = np.column_stack((pred1_final, pred2_final))

In [ ]:
# 최종 예측 수행
final_predictions = meta_model.predict(meta_test_data)

# 정확도 평가
accuracy = accuracy_score(y_test, final_predictions)
print(f'정확도: {accuracy}')

정확도: 1.0
